In [1]:
import os
import cv2
from PIL import Image
import numpy as np

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
image_directory = '/content/drive/My Drive/ML_PROJECTS/'
SIZE = 150
dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.
label = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.

fog_images = os.listdir(image_directory+'Fog')
for i, image_name in enumerate(fog_images):    #Remember enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory +'Fog/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)

#Iterate through all images in Uninfected folder, resize to 64 x 64
#Then save into the same numpy array 'dataset' but with label 1

rain_images = os.listdir(image_directory+'Rain/')
for i, image_name in enumerate(rain_images):
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory +'Rain/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

dataset = np.array(dataset)
label = np.array(label)

print(dataset.shape)
print(label.shape)

(500, 150, 150, 3)
(500,)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.20, random_state = 0)


from tensorflow.keras.utils import normalize
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

In [6]:
INPUT_SHAPE = (SIZE, SIZE, 3)
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
history = model.fit(X_train,
                         y_train,
                         batch_size = 64,
                         verbose = 1,
                         epochs = 30,
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )


model.save(image_directory+'Weather_predictions.h5')

Epoch 1/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 988ms/step - accuracy: 0.6125 - loss: 0.6866 - val_accuracy: 0.5700 - val_loss: 0.6790
Epoch 2/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - accuracy: 0.6319 - loss: 0.6732 - val_accuracy: 0.5600 - val_loss: 0.6656
Epoch 3/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6390 - loss: 0.6504 - val_accuracy: 0.6700 - val_loss: 0.6327
Epoch 4/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7090 - loss: 0.6048 - val_accuracy: 0.6700 - val_loss: 0.5985
Epoch 5/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.7379 - loss: 0.5744 - val_accuracy: 0.7100 - val_loss: 0.5411
Epoch 6/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7266 - loss: 0.5429 - val_accuracy: 0.7200 - val_loss: 0.5472
Epoch 7/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7360 - loss: 0.5739 - val_accuracy: 0.7000 - val_loss: 0.5773
Epoch 8/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7320 - loss: 0.5327 - val_accuracy: 0.7500 - val_loss: 0.53